# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

## Load EMNIST dataset

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

## Preprocessing

In [9]:
def preprocess_emnist(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    image = tf.expand_dims(image, -1)            # Add channel dimension
    label = label - 1                            # Shift labels from 1-26 -> 0-25
    return image, label

ds_train = ds_train.map(preprocess_emnist).batch(32).prefetch(1)
ds_test = ds_test.map(preprocess_emnist).batch(32).prefetch(1)

## Visualize samples

In [ ]:
for image, label in ds_train.take(1):
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(tf.squeeze(image[i]), cmap='gray')
        plt.title(chr(label[i].numpy() + 65))  # Convert to A-Z
        plt.axis('off')
    plt.show()

## Build model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))  # 26 letters

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Train Model

In [ ]:
history = model.fit(ds_train, epochs=10, validation_data=ds_test)

## Save model

In [ ]:
model.save('emnist_model.h5')

## Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(ds_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")